In [1]:
from pandas import read_csv
# load file as CSV
filename = 'train_v2.csv'
mapping_csv = read_csv(filename)
# summarize properties
print(mapping_csv.shape)
print(mapping_csv[:10])

(40479, 2)
  image_name                                         tags
0    train_0                                 haze primary
1    train_1              agriculture clear primary water
2    train_2                                clear primary
3    train_3                                clear primary
4    train_4    agriculture clear habitation primary road
5    train_5                           haze primary water
6    train_6  agriculture clear cultivation primary water
7    train_7                                 haze primary
8    train_8        agriculture clear cultivation primary
9    train_9   agriculture clear cultivation primary road


In [1]:
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed
from pandas import read_csv
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

def create_tag_mapping(mapping_csv):
# create a set of all known tags
    labels = set()
    for i in range(len(mapping_csv)):
# convert spaced separated tags into an array of tags
        tags = mapping_csv['tags'][i].split(' ')
# add tags to the set of known labels
        labels.update(tags)
# convert set of labels to a list to list
    labels = list(labels)
# order set alphabetically
    labels.sort()
# dict that maps labels to integers, and the reverse
    labels_map = {labels[i]:i for i in range(len(labels))}
    inv_labels_map = {i:labels[i] for i in range(len(labels))}
    return labels_map, inv_labels_map

def create_file_mapping(mapping_csv):
    mapping=dict()
    for i in range(len(mapping_csv)):
        names,tags= mapping_csv['image_name'][i],mapping_csv['tags'][i]
        mapping[names]= tags.split(' ')
    return mapping

def one_hot_encode(tags,mapping):
    encoding=zeros(len(mapping),dtype='uint8')
    for tag in tags:
        encoding[mapping[tag]]=1
    return encoding

def create_memory_dataset(path,file_mapping,tag_mapping):
    # i need to store photos and target vectors so create two lists
    photos, targets= list(), list()
    # once the lists are made i need to load the images and convert them to numpy arrays
    for filename in listdir(folder):
        photo= load_img(path+filename, target_size=(32,32))
        photo=img_to_array(photo, dtype='uint8')
        # now i need to create the tags to one hot encode the targets
        tags=file_mapping[filename[:-4]] 
        target= one_hot_encode(tags,tag_mapping)
        photos.append(photo)
        targets.append(target)
    X=asarray(photos, dtype='uint8')
    y = asarray(targets, dtype='uint8')
    return X, y

filename = 'train_v2.csv'        
mapping_csv = read_csv(filename)
# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(mapping_csv)
# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(mapping_csv)
folder = 'train-jpg/'
X, y = create_memory_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)
# save both arrays to one file in compressed format
savez_compressed('Arsal_data2.npz', X, y)

Using TensorFlow backend.


(40479, 32, 32, 3) (40479, 17)


In [2]:
# load prepared planet dataset
from numpy import load
data = load('Arsal_data2.npz')
X, y = data['arr_0'], data['arr_1']
print('Loaded: ', X.shape, y.shape)

Loaded:  (40479, 32, 32, 3) (40479, 17)
